# Are users home locations in Zone A?

In [1]:
import pandas as pd; import numpy as np; from multiprocessing import Pool, Manager; import numpy as np; import geopandas as gpd
import matplotlib.pyplot as plt; import seaborn as sns
import matplotlib, os, json, sys, time, datetime
from bson import json_util

In [2]:
input_directory = "/data/chime/geo/zone_a_full_contexual/stage2"
output_directory = "/data/chime/geo/zone_a_full_contexual/stage3-geovulnerable"

#### Import the ZoneA Geometry

In [3]:
import fiona, shapely; from osgeo import ogr; from shapely.geometry import mapping, shape
c = fiona.open('../ZoneA_Geometry/ZoneA/OGRGeoJSON.shp','r')
pol = c.next(); zoneA = shape(pol['geometry']).buffer(0); zoneA.is_valid

True

#### Import the user metadata DF (Phasing this out)

In [4]:
_user_meta = pd.read_json(input_directory+'/temporal_clustered_user_meta.json')

In [5]:
_user_meta = _user_meta.sort_index()
_user_meta.head(3)

,home_cluster,home_cluster_coords,tweets,uid,user
0,1.0,"{""type"": ""Point"", ""coordinates"": [-73.69254581...",27551,75153082,Andrewthemark
1,NaN,None,17918,450803155,Da_BBCofQU
2,1.0,"{""type"": ""Point"", ""coordinates"": [-74.02219865...",16244,479562736,GinsburgJobs


## 0. Import all of the individual user dataframes

In [6]:
users_in = sorted(os.listdir(input_directory))
users_in = [x for x in users_in if x != "temporal_clustered_user_meta.json"]
print("Found {0} users in {1}".format(len(users_in), input_directory))

Found 954 users in /data/chime/geo/zone_a_full_contexual/stage2


In [9]:
def loader_function(args):
    uFile, path, q = args
    u = json.load(open(path+"/"+uFile,'r'))
    tweets = []
    for t in u['features']:
        if t['geometry']:
            t['properties']['geometry'] = shape(t['geometry'])
        t['properties']['date'] = pd.Timestamp(t['properties']['date'])
        tweets.append(t['properties'])
    q.put(1)
    return gpd.GeoDataFrame(tweets)

In [10]:
#Parallel runtime
p = Pool(30)
m = Manager()
q = m.Queue()

args = [(i, input_directory, q) for i in users_in]
result = p.map_async(loader_function, args)

# monitor loop
while True:
    if result.ready():
        break
    else:
        size = q.qsize()
        sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(size, size/len(args)*100))
        time.sleep(0.5)
sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(q.qsize(), q.qsize()/len(args)*100))
users = result.get()
p.close()

Processed: 954, 100%

In [11]:
users.sort( key=lambda x: len(x), reverse=True ) 

`users` is an array of user dataframes. Now find which users have _home locations_ in Zone A

In [12]:
x = users[100]
print(len(x))
x.head(1)

2560


,cluster,cluster_center,coords,date,day_cluster,geo_delta,geometry,home_cluster_id,speed,text,time_delta,tweet_id,uid,user
0,NaN,null,None,2012-09-02 00:35:34+00:00,11,NaN,NaN,1.0,NaN,simply amazing .,-7781622.0,242058149018869761,295968813,callme_Greishka


In [13]:
def get_home_cluster_center(userDF):
    return shape(json.loads(userDF.query('cluster=='+str(userDF.home_cluster_id.values[0])).cluster_center.values[0]))

get_home_cluster_center(users[0]).within(zoneA)

False

## 1. Determine who's home cluster center is in Zone A

In [14]:
def insideZoneA(p):
    if p==None:
        return False
    else:
        return p.within(zoneA)

In [15]:
vuln = []
non_vuln = []
for idx, u in enumerate(users):
    if (insideZoneA(get_home_cluster_center(u))):
        vuln.append(u)
    else:
        non_vuln.append(u)
    sys.stderr.write("\r"+str(idx+1))
sys.stderr.write("\rDone...")
sys.stderr.write("Identified {0} vulnerable users and {1} non-vulnerable".format(len(vuln),len(non_vuln)))

Done...Identified 90 vulnerable users and 864 non-vulnerable

In [16]:
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

## 2. Write out just the GeoVulnerable, just in case we need them for something later

In [22]:
def safe_mapping(p):
    if p==None or np.isnan(p).any():
        return None
    else:
        return mapping(p)
def safe_json_export(args):
    df, path = args
    df = df.copy()
    uName = df.head(1).user.values[0].lower()
    df['date'] = df['date'].apply(lambda t: datetime.datetime.strftime(t,'%Y-%m-%dT%H:%M:%SZ'))

    clean = df.where((pd.notnull(df)), None)
    geojson = {"type":"FeatureCollection","features":[]}
    for _, row in clean.iterrows():
        geom = safe_mapping(row.geometry)
        feature = {'type':'Feature',
                   'geometry':geom,
                   'properties':row.to_dict()
                    }
        del feature['properties']['geometry']
        geojson['features'].append(feature)
    
    with open(path+"/"+uName+'.geojson','w') as oFile:
        json.dump(geojson, oFile) 

In [23]:
for idx, u in enumerate(vuln):
    safe_json_export((u,output_directory))
    sys.stderr.write("\r{0} processed".format(idx+1))

90 processed

# 3. Prepare for Analysis

All of these users should already exist in a format prepared for visualizing, these users can be pasted into Google Sheets

In [24]:
for uName in sorted([u.user[0].lower() for u in vuln]):
    print(uName)

_an_oak_tree_
acp_clydee
agate86
aladesnr
alliegallo
anas_a1
arnellmilton
bencashfarouq
bridgetekelly
britescrnmedia
carminenyc
cavemanrobles
continuumcycles
dee9996
dem_callmi_tutu
dj2020
dj_vasquezz
dommydom24
eddiegeenyc
edelsingh
emmarocks
fanlyeduardo
g0ttalov3him
garyalonynyc
georgiaafowler
georgieeeninjaa
heatherjeanhd
honeyberk
jaclyn_collins
jamesshowell
japan2021
jcelona_
joemckinzie
jojohankey
jonclarkjr
jonmcl
julianandsara
justmealiseo
katcatoure
kflow94
kmfmack
ladygaga11228
lalahearts
laumc822
laurakazam
lauren_morra
lindseyhankes
lunaparkconeyis
m3lissasays
maderised
marky_mark34
marotin507
mikeviktorenko
moishgeorge86
moneyismotive92
morgansteve
mrspuertorico
muchotoomucho
murphym45
myluvisking
mzmimi_82
newyorkcpt
nickvargas6
nyicecream
nyloves_1d
nzavaa
objorionto
onicadiana
pattyyunen
porcelina1023
princessdds
pullacky
readyrock7
rockawaytrading
rollingstoned40
sarahphara_tw
sfloridia98
shatishahaywood
shes_ayee_shmut
skiftnews
smoshysydney
sotnakny
squish108
tabeek

In [25]:
len(vuln)

90